# Frame Unwrapping

## Default

In this case there is a 1:1 correspondence between source pulses and neutron frames propagated to the sample and detectors.

In [ ]:
from frameunwrapping import default_frames
default_frames().show()

In the figure above, we define for a source frame (pulse) with index `i`

- $T_0^i$ is the `event_time_zero` recorded in an `NXevent_data` group.
  These times are indicated by the vertical dotted lines.
- $T_0^{i+1} = T_0 + L_0$ with the frame length $L_0 = 1/f_0$ given by the source frequency $f_0$.
- $\Delta T_0$ is the offset from $T_0^i$ at which the neutrons are "emitted".
  This may be zero if the full pulse length is used, but choppers such as resolution choppers may extract a section of the pulse that is not aligned with the start of the full pulse.
  This offset can also be used to take into account a potential difference between the timing system's definition of the pulse time and the actual beginning of the neutron pulse exiting, e.g., the moderator.

For every event recorded in a detector, `event_time_offset` ($\Delta t$) in an `NXevent_data` group is the offset from the latest (previous) `event_time_zero` ($T_0^j$), i.e., the time difference to the previous vertical dotted line.

To compute the time-of-flight for a neutron, we need to identify which source pulse it orginated from.
Consider the wavelength band for frame `i`, i.e., neutrons produced by the second source pulse in the figure above:

- The fastest neutrons from this frame reach detector 1 one frame later, i.e., the `event_time_offset` for those fast neutrons will be relative to $T_0^{i+1}$.
- The slowest neutrons from this frame reach detector 1 two frames later, i.e., the `event_time_offset` for those fast neutrons will be relative to $T_0^{i+2}$.
- The fastest neutrons from this frame reach detector 2 two frames later, i.e., the `event_time_offset` for those fast neutrons will be relative to $T_0^{i+2}$.
- The slowest neutrons from this frame reach detector 2 three frames later, i.e., the `event_time_offset` for those fast neutrons will be relative to $T_0^{i+3}$.

We thus need to add an integer multiple of the frame length to `event_time_offset`.
Within a given frame (indicated above by an area between two dotted vertical lines) there is a *pivot point*:
Neutrons with `event_time_offset` *below* the pivot point originated one source frame *before* neutrons *above* the pivot point.
As illustrated in the figure, the pivot point $t_\text{pivot}$ depends on the detector or rather the distance of the detector from the sample.

The pivot point can be computed, e.g., by defining the minimum wavelength $\lambda_{\text{min}}$ that can propagate through the chopper cascade to reach a detector.
The computation of the time-of-flight can then proceed as follows (see also illustration below):

1. Given $\lambda_\text{min}$, we can compute the the corresponding $t_{\text{min}}$ (`tof_min`), i.e., the time the fastest neutrons take from the source to the detector.
   `tof_min` is detector-dependent.
2. The pivot time `time_offset_pivot` is computed as $t_\text{pivot} = (\Delta T_0 + t_{\text{min}})\mod L_0$.
3. The time-of-flight $t_\text{tof}$ is computed as
   $$
   t_{\text{tof}} =  t_{\text{min}} - t_\text{pivot} +\Delta t + 
       \begin{cases}
       L_0, & \text{for } \Delta t \lt t_\text{pivot}\\
       0, & \text{for } \Delta t \ge t_\text{pivot}
       \end{cases}
   $$

In [ ]:
from scippneutron.tof import frames
import scipp as sc
sc.show_graph(frames.to_tof(), simplified=True)

## Wavelength-frame multiplication

## Pulse-skipping